In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.15-py3-none-any.whl size=99605 sha256=1fd390fefd8112eca48aba7ada3b9daf0167b99638c89deaaedddb3f9ef3fdc0
  Stored in directory: /root/.cache/pip/wheels/46/0f/33/40c049c224ee941c2b3a7abb858fc34d93e827f9a833d40f09
Successfully built kaggle


In [ ]:
!pip show kaggle


Name: kaggle
Version: 1.5.15
Summary: Kaggle API
Home-page: https://github.com/Kaggle/kaggle-api
Author: Kaggle
Author-email: support@kaggle.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: bleach, certifi, python-dateutil, python-slugify, requests, six, tqdm, urllib3
Required-by: 


In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jhojanroyqlop","key":"90c5b9e02859df5dba8707ce5d80c8f1"}'}

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                           title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
alphiree/cardiovascular-diseases-risk-prediction-dataset      Cardiovascular Diseases Risk Prediction Dataset       5MB  2023-07-03 12:12:19           2548        111  1.0              
nelgiriyewithana/countries-of-the-world-2023                  Global Country Information Dataset 2023              23KB  2023-07-08 20:37:33           1475         61  1.0              
iamsouravbanerjee/data-science-salaries-2023                  Data Science Salaries                                32KB  2023-07-07 12:53:23           1432         57  1.0              
shuvammandal121/9000-companies-and-its-ratings-dataset        9000 com

In [ ]:
!kaggle --version

Kaggle API 1.5.15


In [ ]:
!pip install kaggle --quiet

In [ ]:
! kaggle datasets download -d spandanpatnaik09/face-mask-detectormask-not-mask-incorrect-mask

 99% 188M/190M [00:08<00:00, 25.3MB/s]
100% 190M/190M [00:08<00:00, 22.7MB/s]


In [ ]:
!unzip /content/face-mask-detectormask-not-mask-incorrect-mask.zip -d "/content/drive/MyDrive/Laboratorios_QuispeLopez/Unidad 2/laboratorio 12"

Archive:  /content/face-mask-detectormask-not-mask-incorrect-mask.zip
  inflating: /content/drive/MyDrive/Laboratorios_QuispeLopez/Unidad 2/laboratorio 12/dataset/incorrect_mask/aug_1.jpg  
  inflating: /content/drive/MyDrive/Laboratorios_QuispeLopez/Unidad 2/laboratorio 12/dataset/incorrect_mask/aug_10.jpg  
  inflating: /content/drive/MyDrive/Laboratorios_QuispeLopez/Unidad 2/laboratorio 12/dataset/incorrect_mask/aug_100.jpg  
  inflating: /content/drive/MyDrive/Laboratorios_QuispeLopez/Unidad 2/laboratorio 12/dataset/incorrect_mask/aug_101.jpg  
  inflating: /content/drive/MyDrive/Laboratorios_QuispeLopez/Unidad 2/laboratorio 12/dataset/incorrect_mask/aug_102.jpg  
  inflating: /content/drive/MyDrive/Laboratorios_QuispeLopez/Unidad 2/laboratorio 12/dataset/incorrect_mask/aug_103.jpg  
  inflating: /content/drive/MyDrive/Laboratorios_QuispeLopez/Unidad 2/laboratorio 12/dataset/incorrect_mask/aug_104.jpg  
  inflating: /content/drive/MyDrive/Laboratorios_QuispeLopez/Unidad 2/laborator

In [ ]:
from __future__ import division
from __future__ import print_function

In [ ]:
!pip install torch torchvision

In [ ]:
import copy
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, models, transforms
from datetime import datetime
from pathlib import Path
print("PyTorch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)

PyTorch Version:  2.0.1+cu118
Torchvision Version:  0.15.2+cu118


In [ ]:
def train_model(model, dataloaders, criterion, optimizer, save_path, num_epochs=25, is_inception=False):
    Path(save_path).mkdir(parents=True, exist_ok=True)
    since = time.time()
    acc_train_history = []
    loss_train_history = []
    acc_val_history = []
    loss_val_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Cada época tiene una fase de entrenamiento y validación
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() # Establecer modelo en modo de entrenamiento
            else:
                model.eval() # Establecer modelo en modo de evaluación

            running_loss = 0.0
            running_corrects = 0.0

            # Repetir los datos
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Cero los gradientes de los parámetros
                optimizer.zero_grad()

                # Reenviar la historia de gradiente aunque solo esté en el entrenamiento
                with torch.set_grad_enabled(phase == 'train'):
                    # Obtener salidas del modelo y calcular la pérdida
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4 * loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # Retroalimentación + optimizar solo si está en fase de entrenamiento
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Estadísticas
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # Copia profunda del modelo
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(), 'loss': loss},
                           f'{save_path}/best.pt')

            if phase == 'val':
                loss_val_history.append(epoch_loss)
                acc_val_history.append(epoch_acc)

            if phase == 'train':
                loss_train_history.append(epoch_loss)
                acc_train_history.append(epoch_acc)

            if epoch % 10 == 0:
                torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(), 'loss': loss},
                           f'{save_path}/model_{epoch}.pt')

        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(), 'loss': loss},
                   f'{save_path}/last.pt')

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    #trazando
    N = num_epochs

    # Plot training accuracy and validation accuracy
    plt.style.use("ggplot")
    fig = plt.figure()
    plt.plot(np.arange(0, N), acc_train_history, label="train_acc")
    plt.plot(np.arange(0, N), acc_val_history, label="val_acc")
    plt.title("Training Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(loc="lower right")
    fig.savefig(f'Accuracy_{model_name}.png')

    # Plot training loss and validation loss
    plt.style.use("ggplot")
    fig = plt.figure()
    plt.plot(np.arange(0, N), loss_train_history, label="train_loss")
    plt.plot(np.arange(0, N), loss_val_history, label="val_loss")
    plt.title("Training Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(loc="upper right")
    fig.savefig(f'Loss_{model_name}.png')
    # cargar pesos al mejor modelo
    model. load_state_dict(best_model_wts)
    return model, acc_val_history

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Inicialice estas variables que se establecerán en esta declaración if. Cada una de estas variables es específica de
    model_ft = None
    input_size = 0

    if model_name == "AlexNet":
        """ Alexnet """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "SqueezeNet":
        """ SqueezeNet """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224
    elif model_name == "DenseNet":
        """ DenseNet """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "Inceptionv3":
        """ Inceptionv3 Be careful, expects (299,299) sized images and has auxiliary output """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Manejar la red auxiliar
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Manejar la red primaria
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 299
    else:
        print("Nombre de modelo inválido, saliendo....")
        exit()

    return model_ft, input_size


In [ ]:
# Directorio de datos de nivel superior. Aquí asumimos que el formato del directorio se ajusta a la estructura ImageFolder.
data_dir = "/content/drive/MyDrive/Laboratorios_QuispeLopez/Unidad 2/laboratorio 12/dataset"
# Modelos para elegir [AlexNet, VGG11-BN, SqueezeNet, DenseNet, Inceptionv3].
model_name = "AlexNet"
# Número de clases en el dataset.
num_classes = 3
# Tamaño de lote para entrenamiento (cambia según la cantidad de memoria que tengas).
batch_size = 8
# Número de épocas para entrenar.
num_epochs = 100
mode = 'test'
# Marcar para extraer características. Cuando es False, ajustamos todo el modelo, cuando es True, solo
# actualizamos los parámetros de capa remodelados.
feature_extract = False

# Inicializar el modelo para esta ejecución.
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Imprime el modelo que acabamos de crear.
print(model_ft)

# Aumento y normalización de datos para el entrenamiento, solo normalización para la validación.
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Detectar si tenemos una GPU disponible.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Enviar el modelo a la GPU.
model_ft = model_ft.to(device)
print("Initializing Datasets and Dataloaders...")

# Crear dataset de entrenamiento y validación.
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Crear dataloaders de entrenamiento y validación.
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=0)
                    for x in ['train', 'val']}

# Retener los parámetros que se optimizarán / actualizarán en esta ejecución. Si estamos afinando, actualizaremos todos
# los parámetros. Sin embargo, si estamos utilizando el método de extracción de características, solo actualizaremos
# los parámetros que acabamos de inicializar, es decir, los parámetros con require_grad son True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name, param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t", name)
else:
    for name, param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t", name)

optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
# Configurar la pérdida fxn.
criterion = nn.CrossEntropyLoss()
save_path = datetime.now().strftime("%d-%m-%Y_%H%M")
# Entrenar y evaluar.
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, save_path=save_path,
                             num_epochs=num_epochs, is_inception=(model_name == "inception"))


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

FileNotFoundError: ignored